In [1]:
%cd ../
%matplotlib inline

/home/eli/AnacondaProjects/HTFATorch


In [2]:
import htfa_torch.niidb as niidb
import htfa_torch.utils as utils

import csv
import glob
import logging
import os
import re

In [3]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

db_file = 'data/synthetic_data.db'
mask_file = 'data/synthetic_data/nifti_files/wholebrain.nii.gz'

OVERWRITE = True

In [4]:
existed = os.path.isfile(db_file)
db = niidb.FMriActivationsDb(db_file, mask=mask_file)

if not existed or OVERWRITE:
    data_files = glob.glob('data/synthetic_data/nifti_files/*.nii')
    regex = re.compile("subject_([0-9]{1,2})_task_([1,2]).nii")
    for data_file in data_files:
        m = regex.search(data_file)
        [subject, task] = m.groups()
        subject = int(subject)
        logging.info("Recording blocks in %s", data_file)
        
        synthetic_block = niidb.FMriActivationBlock(zscore=False)
        synthetic_block.task = task
        synthetic_block.filename = data_file
        synthetic_block.subject = subject
        synthetic_block.start_time = 0
        synthetic_block.end_time = None
        db.upsert(synthetic_block)
        logging.info('Subject %d performing task %s', subject, synthetic_block.task)

01/07/2019 15:00:17 Recording blocks in data/synthetic_data/nifti_files/subject_5_task_1.nii
01/07/2019 15:00:17 Subject 5 performing task 1
01/07/2019 15:00:17 Recording blocks in data/synthetic_data/nifti_files/subject_30_task_2.nii
01/07/2019 15:00:17 Subject 30 performing task 2
01/07/2019 15:00:17 Recording blocks in data/synthetic_data/nifti_files/subject_30_task_1.nii
01/07/2019 15:00:17 Subject 30 performing task 1
01/07/2019 15:00:17 Recording blocks in data/synthetic_data/nifti_files/subject_22_task_2.nii
01/07/2019 15:00:17 Subject 22 performing task 2
01/07/2019 15:00:17 Recording blocks in data/synthetic_data/nifti_files/subject_17_task_2.nii
01/07/2019 15:00:17 Subject 17 performing task 2
01/07/2019 15:00:17 Recording blocks in data/synthetic_data/nifti_files/subject_20_task_2.nii
01/07/2019 15:00:17 Subject 20 performing task 2
01/07/2019 15:00:17 Recording blocks in data/synthetic_data/nifti_files/subject_6_task_2.nii
01/07/2019 15:00:17 Subject 6 performing task 2
01/

In [5]:
logging.info('Finished building NiiDb out of synthetic dataset')

01/07/2019 15:00:20 Finished building NiiDb out of synthetic dataset
